# Basic Library

In [1]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None
import numpy as np
 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set('talk')
sns.set_style('white')

# Check for GPU

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are {} GPU(s) available.'.format(torch.cuda.device_count()))

    print('We will use the GPU: {}'.format(torch.cuda.get_device_name(0)))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

# Helper Function

In [ ]:
import os

__print__ = print
# Function to print statements into Kaggle logs
def print(string):
    os.system(f'echo \"{string}\"')
    __print__(string)

# Import Data

In [ ]:
data = pd.read_csv('../input/sentimenttranslated/train_translated.csv')

In [ ]:
sns.countplot(x='rating', data=data);

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-large-uncased'
MAX_LENGTH = 40
RANDOM_SEED = 2048
BATCH_SIZE = 32
EPOCHS = 10

DEBUG = False

In [ ]:
# For debugging purposes
if DEBUG:
    data = data.sample(n=1000)

# Import Bert Tokenizer

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
token_length = []

# For every review...
for review in data['review'].values:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(review, add_special_tokens=True)
    
    # Append to list
    token_length.append(len(input_ids))

print('Max sentence length: {}'.format(np.max(token_length)))

In [ ]:
# Check distribution of token length
sns.distplot(token_length)

plt.xlim([0, 100]);

plt.xlabel('Token count');

# Train Validation Split

In [ ]:
from sklearn.model_selection import train_test_split

# Convert rating from 0 to 4
data['rating'] = data['rating'] - 1

df_train, df_val = train_test_split(data, test_size=0.2, random_state=RANDOM_SEED, stratify=data['rating'])

In [ ]:
def tokenize(df, test=False):
    
    # Tokenize all of the sentences and map the tokens to their word IDs.
    input_ids = []
    attention_masks = []
    
    reviews = df['review'].values
    
    # For every review...
    for review in reviews:
        # `encode_plus` will:
        #   (1) Tokenize the review.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            review,                       # Sentence to encode.
                            add_special_tokens = True,    # Add '[CLS]' and '[SEP]'
                            max_length = MAX_LENGTH,      # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True, # Construct attn. masks.
                            return_tensors = 'pt',        # Return pytorch tensors.
                            truncation=True
                       )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    if not test:
        ratings = df['rating'].values
        ratings = torch.tensor(ratings, dtype=torch.long)
        print('Original: {}'.format(reviews[0]))
        print('Token IDs: {}'.format(input_ids[0]))
        print('')
        return input_ids, attention_masks, ratings
    
    # Print sentence 0, now as a list of IDs.
    print('Original: {}'.format(reviews[0]))
    print('Token IDs: {}'.format(input_ids[0]))
    print('')
    
    return input_ids, attention_masks

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_input_ids, train_attention_masks, train_ratings = tokenize(df_train)
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_ratings)

# Combine the validation inputs into a TensorDataset.
val_input_ids, val_attention_masks, val_ratings = tokenize(df_val)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_ratings)

print('{:>5,} training samples'.format(df_train.shape[0]))
print('{:>5,} validation samples'.format(df_val.shape[0]))

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Number of classes
classes = np.unique(df_train['rating'])
num_classes = len(classes)

# Calculate weight for each label
weights = compute_class_weight(class_weight='balanced', classes=classes, y=df_train['rating'])

# Assign weight to each data point
df_train['weight'] = df_train['rating'].map(dict(enumerate(weights)))
df_val['weight'] = df_val['rating'].map(dict(enumerate(weights)))

train_weights = df_train['weight'].values
val_weights = df_val['weight'].values

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = WeightedRandomSampler(torch.DoubleTensor(train_weights), int(df_train.shape[0]), replacement=False),
            batch_size = BATCH_SIZE, # Trains with this batch size.
            num_workers = 6 # Number of workers
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = WeightedRandomSampler(torch.DoubleTensor(val_weights), int(df_val.shape[0]), replacement=False),
            batch_size = BATCH_SIZE, # Evaluate with this batch size.
            num_workers = 6 # Number of workers
        )

# Import Bert Model

In [ ]:
from transformers import BertModel, AdamW, BertConfig
from torch import nn

class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
#     self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    # last_hidden_state (BATCH_SIZE, MAX_LENGTH, BERT.CONFIG.HIDDEN_SIZE)
    # pooled_output (BATCH_SIZE, BERT.CONFIG.HIDDEN_SIZE): [CLS] -> linear + tanh activation
    
    last_hidden_state, pooled_output = self.bert(input_ids=input_ids,
                                                 attention_mask=attention_mask)
#     output = self.drop(pooled_output)
    output = self.out(pooled_output)
    return output, last_hidden_state[:, 0, :]

model = SentimentClassifier(num_classes)

# Tell pytorch to run this model on the GPU.
model.cuda()

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

# Import Optimizer

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * EPOCHS

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

loss_fn = nn.CrossEntropyLoss(weight=torch.from_numpy(weights).float()).to(device)

# Training 

In [ ]:
from sklearn.metrics import balanced_accuracy_score

# Function to calculate balanced accuracy of our predictions vs labels
def metric(logits, labels):
    preds_flat = np.argmax(logits, axis=1).flatten()
    labels_flat = labels.flatten()
    return balanced_accuracy_score(labels_flat, preds_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
def good_update_interval(total_iters, num_desired_updates):
    '''
    This function will try to pick an intelligent progress update interval 
    based on the magnitude of the total iterations.

    Parameters:
      `total_iters` - The number of iterations in the for-loop.
      `num_desired_updates` - How many times we want to see an update over the 
                              course of the for-loop.
    '''
    # Divide the total iterations by the desired number of updates. Most likely
    # this will be some ugly number.
    exact_interval = total_iters / num_desired_updates

    # The `round` function has the ability to round down a number to, e.g., the
    # nearest thousandth: round(exact_interval, -3)
    #
    # To determine the magnitude to round to, find the magnitude of the total,
    # and then go one magnitude below that.

    # Get the order of magnitude of the total.
    order_of_mag = len(str(total_iters)) - 1

    # Our update interval should be rounded to an order of magnitude smaller. 
    round_mag = order_of_mag - 1

    # Round down and cast to an int.
    update_interval = int(round(exact_interval, -round_mag))

    # Don't allow the interval to be zero!
    if update_interval == 0:
        update_interval = 1

    return update_interval

In [ ]:
import random
import torch.nn.functional as F

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Update every `update_interval` batches.
update_interval = good_update_interval(total_iters=len(train_dataloader), num_desired_updates=5)

# Measure the total training time for the whole run.
total_t0 = time.time()

# Track best val acc
best_val_acc = 0

# Track best val loss
best_val_loss = 0

# For each epoch...
for epoch_i in range(0, EPOCHS):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, EPOCHS))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every batches.
        if step % update_interval == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # are given and what flags are set. For our usage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        
        logits, embeddings = model(b_input_ids,
                                   attention_mask=b_input_mask)
        
        loss = loss_fn(logits, b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.3f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits, embeddings = model(b_input_ids,
                                       attention_mask=b_input_mask)

        loss = loss_fn(logits, b_labels)

        # Accumulate the validation loss.
        total_eval_loss += loss.item()
        
        # Move preds and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        
        acc = metric(logits, label_ids)
        total_eval_accuracy += acc
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.3f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.3f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
    
    # Save model
    if avg_val_accuracy > best_val_acc:
        torch.save(model.state_dict(), 'best_val_acc_state.pth')
        best_val_acc = avg_val_accuracy
    
    if avg_val_loss > best_val_loss:
        torch.save(model.state_dict(), 'best_val_loss_state.pth')
        best_val_loss = avg_val_loss

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

# Training Statistics

In [ ]:
# Display floats with three decimal places.
pd.set_option('precision', 3)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

# Evaluate on Test

In [ ]:
df_test = pd.read_csv('../input/sentimenttranslated/test_translated.csv')

if DEBUG:
    df_test = df_test.sample(n=1000)

# Combine the test inputs into a TensorDataset.
test_input_ids, test_attention_masks = tokenize(df_test, test=True)
test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=BATCH_SIZE, num_workers=6)

print('{:>5,} training samples'.format(df_test.shape[0]))

In [ ]:
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(test_input_ids)))

# Load best model
model = SentimentClassifier(num_classes)
model.load_state_dict(torch.load('best_val_loss_state.pth'))
model.cuda()

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions = []
test_embeddings = []

# Predict 
for batch in test_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits, embeddings = model(b_input_ids,
                                 attention_mask=b_input_mask)
    
    _, preds = torch.max(logits, dim=1)
    
    probs = F.softmax(logits, dim=1)
    
    # Move logits and labels to CPU
    probs = probs.detach().cpu().numpy()
    embeddings = embeddings.detach().cpu().numpy()

    # Store probabilities
    predictions.append(probs)
    
    #Store embeddings
    test_embeddings.append(np.vstack(embeddings))
    
print('    DONE.')

In [ ]:
# Combine the results across all batches. 
flat_probabilities = np.concatenate(predictions, axis=0)

# For each sample, pick the label with the higher score.
# flat_predictions = np.argmax(flat_probabilities, axis=1).flatten()

kaggle = df_test[['review_id']]

for i in range(num_classes):
    kaggle['probability_{}'.format(i)] = flat_probabilities[:, i]

kaggle.to_csv('probabilities.csv', index=False)

# Save embeddings
flat_test_embeddings = np.concatenate(test_embeddings, axis=0)
np.save('test_embeddings.npy', flat_test_embeddings)

# Embeddings for Original Data

In [ ]:
# Tracking variables 
train_embeddings = []

# Embeddings for train
print('Embeddings for data...')

# Combine the full data inputs into a TensorDataset.
full_input_ids, full_attention_masks, full_ratings = tokenize(data)
full_dataset = TensorDataset(full_input_ids, full_attention_masks, full_ratings)
full_dataloader = DataLoader(full_dataset, sampler=SequentialSampler(full_dataset), batch_size=BATCH_SIZE, num_workers=6)

# Put model in evaluation mode
model.eval()

# Predict 
for batch in full_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits, embeddings = model(b_input_ids,
                                 attention_mask=b_input_mask)
        
    embeddings = embeddings.detach().cpu().numpy()
    
    #Store embeddings
    train_embeddings.append(np.vstack(embeddings))
    
print('    DONE.')

In [ ]:
# Save embeddings
flat_train_embeddings = np.concatenate(train_embeddings, axis=0)
np.save('train_embeddings.npy', flat_train_embeddings)